In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,Dense,Dropout, Reshape, Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

from tensorflow.keras.optimizers import Adam

from gensim.models import Word2Vec
import gensim

import ast
import nltk

In [2]:
from google.cloud import storage
import os
import io
client = storage.Client()
bucket = client.get_bucket('nlp_final_data')

blob = bucket.blob('data_no_stop.csv')
content = blob.download_as_string()

df = pd.read_csv(io.BytesIO(content))

In [ ]:
# data is saved a string, convert it back to a list of strings
df['tokens_no_stop'] = df['tokens_no_stop'].apply(lambda x: x.strip('[]').replace('\'', '').split(', '))

In [8]:
from nltk.corpus import words

nltk.download('wordnet')
lemmatizer = nltk.WordNetLemmatizer()

nltk.download('words')
english_dictionary = set(words.words())

def clean_tokens(words):
#     # Remove single-character tokens (mostly punctuation)
#     words = [word for word in words if len(word) > 1]

#     # Remove numbers
#     words = [word for word in words if not word.isnumeric()]

#     # Remove punctuation
#     words = [word for word in words if word.isalpha()]

    # Lowercase all words (default_stopwords are lowercase too)
    words = [word.lower() for word in words]
    
    words = [word for word in words if word.lower() in english_dictionary]
    
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /home/jupyter/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [45]:
df['1-grams'] = df['tokens_no_stop'].apply(lambda x: clean_tokens(x))

In [3]:
df['1-grams'] = df['1-grams'].apply(lambda x: x.strip('[]').replace('\'', '').split(', '))

In [50]:
for i in range(2,3):
    col_name = str(i)+'-grams'
    df[col_name] = df['1-grams'].apply(lambda x: list(nltk.ngrams(x,i)))

In [53]:
df[['tag', '1-grams','2-grams', '3-grams']].to_csv('songs_nGrams.csv')

In [2]:
from google.cloud import storage
import os
import io
client = storage.Client()
bucket = client.get_bucket('nlp_final_data')

blob = bucket.blob('songs_nGrams.csv')

In [ ]:
blob = bucket.blob('songs_nGrams.csv')
blob.upload_from_filename('songs_nGrams.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'songs_nGrams.csv'

In [72]:
df = pd.read_csv('songs_nGrams.csv')

In [35]:
def tuple_to_list(tuple_list):
    strings = []
    for l in tuple_list:
        ngrams = []
        for ngram in l:
            ngram_string = ' '.join(str(element) for element in ngram)
            ngrams.append(ngram_string)
            print(ngrams)
        strings.append(ngrams)
    return strings

In [39]:
test = tuple_to_list(df['2-grams'][:3])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [51]:
#df['3-grams_string'] = df['3-grams'].apply(lambda x: [' '.join(str(element) for element in tuple_) for tuple_ in x])
df['2-grams_string'] = df['2-grams'].apply(lambda x: [' '.join(str(element) for element in tuple_) for tuple_ in x])

In [58]:
grams1 = df['1-grams']
grams2 = df['2-grams_string']
grams3 = df['3-grams_string']

combined_grams = []
for i in range(len(grams1)):
    combined_grams.append(grams1[i]+grams2[i])

In [59]:
df['up_to_bigrams'] = combined_grams

In [ ]:
combined_grams = []
for i in range(len(grams1)):
    combined_grams.append(grams1[i]+grams2[i]+grams3[i])

In [ ]:
df['up_to_trigrams'] = combined_grams

In [61]:
df[['tag','up_to_trigrams', 'up_to_bigrams']].to_csv('songs_combined_ngrams.csv')

In [64]:
blob = bucket.blob('songs_combined_ngrams.csv')
blob.upload_from_filename('songs_combined_ngrams.csv')

In [73]:
X = list(df['1-grams'])
y = df['tag']

In [ ]:
# https://spotintelligence.com/2023/02/15/word2vec-for-text-classification/

In [74]:
# split the balanced data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2,stratify=y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [75]:
tokenize = Tokenizer(char_level=False)
tokenize.fit_on_texts(X)

# we will need this later
num_words = len(tokenize.word_index)+1

In [76]:
#num_words is 496351 with just tokens, and not cleaned
#num_words is 46207 when data is lemmatized and cleaned
num_words

46207

In [77]:
x_train = tokenize.texts_to_sequences(X_train)
x_test = tokenize.texts_to_sequences(X_test)

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [78]:
max_log_length = 1024
x_train = pad_sequences(x_train, maxlen=max_log_length)
x_test = pad_sequences(x_test, maxlen=max_log_length)

In [79]:
model3 = Sequential()

model3.add(Embedding(input_dim=num_words,output_dim=64,input_length=max_log_length))

num_filters = 128
kernel_sizes = [2,3]
for kernel_size in kernel_sizes:
    model3.add(Conv1D(num_filters, kernel_size, activation='relu'))
#model3.add(Conv1D(128, 3, activation='relu'))

model3.add(MaxPooling1D(pool_size=2))
model3.add(LSTM(units=64,recurrent_dropout=0.5))
model3.add(Dropout(0.5))

# Add Dense layers
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.5))
    
model3.add(Dense(num_classes,activation='softmax'))

# compile the model
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [80]:
history = model3.fit(x_train, y_train,
                    batch_size=128,
                    epochs=3,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/3
2438/2438 [==============================] - 1640s 672ms/step - loss: 1.1686 - accuracy: 0.5217 - val_loss: 1.0548 - val_accuracy: 0.5761
Epoch 2/3
2438/2438 [==============================] - 1640s 673ms/step - loss: 1.0396 - accuracy: 0.5836 - val_loss: 1.0421 - val_accuracy: 0.5757
Epoch 3/3
1284/2438 [==============>...............] - ETA: 12:40 - loss: 0.9887 - accuracy: 0.6041

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2438/2438 [==============================] - 1647s 676ms/step - loss: 0.9902 - accuracy: 0.6043 - val_loss: 1.0358 - val_accuracy: 0.5814


In [81]:
model3.save('model_0519.h5')

In [82]:
blob_name = 'model_0519.h5'
client = storage.Client()

blob = bucket.blob(blob_name)
blob.upload_from_filename(blob_name)

Using convolutional layers in a neural network for text data can be beneficial for several reasons:

Local Pattern Extraction: Convolutional layers can effectively capture local patterns and features in the text data. By applying filters of different sizes, the convolutional operation can detect patterns at various levels of granularity. This allows the model to learn relevant features such as n-grams, word combinations, or other local patterns that are indicative of the text's meaning or sentiment.

Translation Invariance: Convolutional layers exhibit translation invariance, which means they can recognize patterns regardless of their exact position in the input. In the context of text data, this property is useful because the position of a particular word or phrase in a sentence may not always be critical for understanding its meaning. By capturing patterns irrespective of their location, convolutional layers can provide robust representations that are not overly sensitive to word order.

Reduced Parameter Count: Convolutional layers can help reduce the number of parameters in the model compared to fully connected layers. This reduction is achieved by weight sharing through the use of filters. By sharing weights, the model can capture the same pattern or feature across different positions in the input, resulting in fewer trainable parameters. This parameter efficiency can make the model easier to train and less prone to overfitting, especially when dealing with limited amounts of text data.

Hierarchical Feature Learning: Deep architectures with multiple convolutional layers can learn hierarchical representations of the input text. Lower-level convolutional layers can capture basic local features, while higher-level convolutional layers can learn more complex combinations of these features. This hierarchical learning enables the model to capture both low-level and high-level semantic information from the text.

It's worth noting that while convolutional neural networks (CNNs) have primarily been associated with image processing tasks, they have been successfully adapted for natural language processing (NLP) tasks, including text classification. The convolutional operations in text CNNs are typically performed along the time dimension (i.e., word or character sequences) rather than across spatial dimensions (as in image CNNs).

That being said, the effectiveness of using convolutional layers in text classification tasks may vary depending on the specific dataset and problem. It's recommended to experiment with different architectures and compare the performance with other approaches, such as recurrent neural networks (RNNs) or transformers, to determine the best choice for your particular task.






Regenerate response